## Transformer
- DETR의 Transformer architecture를 기반으로 함.
- 차이점은 아래와 같다.
- 1. positional encoding이 Multi-Head Attention에 직접 전달
- 2. Encoder의 마지막 LayerNorm이 제거되었다.
- 3. Decoder가 여러 Layer의 출력을 모두 반환할 수 있도록 변경
- 4. 이전 Frame 정보를 활용할 수 있도록 Transformer architecture 확장 (track_attention=True)

In [ ]:
import copy
from typing import Optional

import torch
import torch.nn.functional as F
from torch import Tensor, nn

`Transformer`
- CNN에서 추출한 Feature을 Transforemr Encoder에서 처리
- Decoder에서 Query 기반 객체 예측 및 Tracking-by-Attention 적용
- 이전 Frame 정보를 활용해 Tracking-by-Attention 수행

In [ ]:
class Transformer(nn.Module):

    def __init__(self, d_model=512, nhead=8, num_encoder_layers=6,
                 num_decoder_layers=6, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False,
                 return_intermediate_dec=False,
                 track_attention=False):
        super().__init__()

        # d_model: Transformer의 차원 수 (512)
        # nhead: Multi-Head Attention의 헤드 수 (8개)
        # num_encoder_layers: Encoder 층 개수 (6개)
        # num_decoder_layers: Decoder 층 개수 (6개)
        # dim_feedforward: Feedforward 네트워크의 차원 수 (2048)
        # dropout: Dropout 비율 (0.1)
        # activation: 활성화 함수 선택 (ReLU)
        # normalize_before: Layer Normalization을 수행하는 위치 선택
        # return_intermediate_dec: Decoder의 중간 출력을 반환할지 여부
        # track_attention: 이전 프레임 정보를 활용하는 Attention 적용 여부
        # => Object Tracking을 위해 track_attention을 추가하여 이전 Frame 정보를 활용용

        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward,
                                                dropout, activation, normalize_before)
        encoder_norm = nn.LayerNorm(d_model) if normalize_before else None
        self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward,
                                                dropout, activation, normalize_before)
        decoder_norm = nn.LayerNorm(d_model)
        self.decoder = TransformerDecoder(
            decoder_layer, encoder_layer, num_decoder_layers, decoder_norm,
            return_intermediate=return_intermediate_dec,
            track_attention=track_attention)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

        # normalize_before: True일 경우 LayerNorm을 Encoder의 입력으로 적용 (Pre-Norm 방식)
        # track_attention=True이면 이전 프레임 정보를 활용하는 추가 Attention Layer를 사용
        # => "Tracking-by-Attention"을 구현하기 위해 Transformer 구조를 확장

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    # => 모든 가중치를 Xavier Uniform 초기화 방식으로 설정

    def forward(self, src, mask, query_embed, pos_embed, tgt=None, prev_frame=None):
        # flatten NxCxHxW to HWxNxC
        bs, c, h, w = src.shape
        src = src.flatten(2).permute(2, 0, 1)
        pos_embed = pos_embed.flatten(2).permute(2, 0, 1)
        mask = mask.flatten(1)

        # src: CNN 백본에서 추출한 특징 맵
        # mask: 패딩(mask) 정보
        # query_embed: 객체를 예측하기 위한 Query Embedding
        # pos_embed: Transformer의 위치 정보를 유지하는 Position Embedding
        # tgt: Decoder의 초기 입력 값 (객체 위치 예측을 위한 Query로 활용됨)
        # prev_frame: 이전 프레임 정보를 저장하여 Tracking 수행 (선택적)
        # => N x C x H x W → HW x N x C 형태로 변환하여 Transformer에 입력할 수 있도록 조정

        if tgt is None:
            tgt = torch.zeros_like(query_embed)
        memory = self.encoder(src, src_key_padding_mask=mask, pos=pos_embed)
        
        # memory: Transformer Encoder에서 추출된 최종 표현 (Feature Memory)
        # => Transformer Encoder는 CNN의 Feature를 입력받아 객체의 시각적 정보를 학습하는 역할

        memory_prev_frame = None
        if prev_frame is not None:
            src_prev_frame = prev_frame['src'].flatten(2).permute(2, 0, 1)
            pos_embed_prev_frame = prev_frame['pos'].flatten(2).permute(2, 0, 1)
            mask_prev_frame = prev_frame['mask'].flatten(1)

            memory_prev_frame = self.encoder(
                src_prev_frame, src_key_padding_mask=mask_prev_frame, pos=pos_embed_prev_frame)

            prev_frame['memory'] = memory_prev_frame
            prev_frame['memory_key_padding_mask'] = mask_prev_frame
            prev_frame['pos'] = pos_embed_prev_frame

        # prev_frame이 존재하면, 이전 프레임의 특징을 Encoder에 다시 전달하여 Memory 저장
        # memory_prev_frame을 Transformer Decoder에 전달하여 Tracking-by-Attention 구현
        # => 즉, 이전 Frame의 정보를 활용하는 것이 TrackFormer의 중요한 차별점

        hs, hs_without_norm = self.decoder(tgt, memory, memory_key_padding_mask=mask,
                                           pos=pos_embed, query_pos=query_embed,
                                           prev_frame=prev_frame)

        # memory: Transformer Encoder에서 나온 Feature Memory
        # query_embed: 객체를 탐색할 Query
        # prev_frame: 이전 Frame 정보를 활용하여 Tracking 수행
        # => Decoder는 현재 Frame의 Feature와 Query를 사용하여 객체를 탐지하고, 이전 Frame 정보를 활용하여 Tracking을 수행

        return (hs.transpose(1, 2),
            hs_without_norm.transpose(1, 2),
            memory.permute(1, 2, 0).view(bs, c, h, w))
    
        # hs: Decoder의 최종 출력 (객체 탐지 결과)
        # memory: Transformer Encoder에서 나온 Feature Memory
        # memory.permute(1, 2, 0).view(bs, c, h, w): CNN Feature Map 크기로 변환하여 반환
        # => 최종적으로 Object Detection 결과를 출력력

`TransformerEncoder`
- CNN에서 추출한 Feature를 Transformer 입력으로 변환
- Encoder를 통해 객체 간의 관계를 학습
- Encoder가 Frame별 특징을 학습

In [ ]:
class TransformerEncoder(nn.Module):

    def __init__(self, encoder_layer, num_layers, norm=None):
        super().__init__()
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    # normalize_before=True이면 LayerNorm을 Encoder의 입력에 적용 (Pre-Norm)
    # normalize_before=False이면 LayerNorm을 Encoder의 출력에 적용 (Post-Norm)   
    # => 여러개의 Transformer Encoder Layer를 쌓아서 입력 특징을 처리리

    def forward(self, src,
                mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None):
        output = src
        # CNN 백본에서 추출한 특징을 Transformer Encoder의 입력으로 전달

        for layer in self.layers:
            output = layer(output, src_mask=mask,
                           src_key_padding_mask=src_key_padding_mask, pos=pos)
        # Multi-Head Attention을 통해 입력 특징 간의 관계 학습

        if self.norm is not None:
            output = self.norm(output)

        return output
    
    # src: CNN 백본에서 추출된 특징 맵 (Flatten된 형태)
    # mask: Attention Mask (필요한 경우만 사용)
    # src_key_padding_mask: 입력에서 패딩된 부분을 무시하기 위한 Mask
    # pos: 위치 인코딩 (Positional Encoding)
    # => Encoder는 CNN 특징을 Transformer 구조로 변환하여 객체 간 관계를 학습하고, 위치 정보를 유지하기 위해 Positional Encoding을 Attention에 직접 전달

`TransformerDecoder`
- Query 기반 객체 탐지 및 Tracking-by-Attention 수행
- 이전 Frame 정보를 활용하여 객체 연속성을 유지 (Multi-Frame Attention)
- Query 일부를 Tracking Query로 변형하여 Frame 간 연관성을 학습
- 객체 추적을 위한 Query Manipulation

In [ ]:
class TransformerDecoder(nn.Module):

    def __init__(self, decoder_layer, encoder_layer, num_layers,
                 norm=None, return_intermediate=False, track_attention=False):
        super().__init__()
        self.layers = _get_clones(decoder_layer, num_layers)

        self.num_layers = num_layers
        self.norm = norm
        self.return_intermediate = return_intermediate

        self.track_attention = track_attention
        if self.track_attention:
            self.layers_track_attention = _get_clones(encoder_layer, num_layers)

    # return_intermediate: Decoder의 중간 출력을 반환할지 여부
    # track_attention: 이전 Frame 정보를 활용할지 여부 

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None,
                prev_frame: Optional[dict] = None):
        
                # tgt: Decoder의 초기 입력 값 (Query Embeddings)
                # memory: Transformer Encoder의 출력 (CNN 특징을 Transformer로 변환한 Feature Memory)
                # tgt_mask: Decoder의 Self-Attention Mask
                # memory_mask: Encoder-Decoder Attention Mask
                # tgt_key_padding_mask: Query에서 패딩된 부분을 무시하기 위한 Mask
                # memory_key_padding_mask: Memory에서 패딩된 부분을 무시하기 위한 Mask
                # pos: Transformer Encoder의 위치 인코딩 (Positional Encoding)
                # query_pos: Query 위치 인코딩
                # prev_frame: 이전 Frame의 정보가 포함된 딕셔너리 (Tracking-by-Attention 수행 시 사용)
                        
        output = tgt

        intermediate = []

        if self.track_attention:
            track_query_pos = query_pos[:-100].clone()
            query_pos[:-100] = 0.0

        # Query Manipulation
        # self.track_attention=True이면, Query 중 일부를 이전 Frame 객체 정보를 추적하는 Query로 활용
        # query_pos[:-100] = 0.0 → Tracking Query가 아닌 부분은 0으로 초기화하여 Tracking Attention에 영향을 주지 않도록 설정
        # => 즉, Query 중 일부는 현재 Frame에서 객체를 탐색하고, 일부는 이전 Frame 객체를 추적하는 데 사용

        for i, layer in enumerate(self.layers):
            if self.track_attention:
                track_output = output[:-100].clone()

                track_output = self.layers_track_attention[i](
                    track_output,
                    src_mask=tgt_mask,
                    src_key_padding_mask=tgt_key_padding_mask,
                    pos=track_query_pos)

                output = torch.cat([track_output, output[-100:]])

        # track_output = output[:-100].clone() → 현재 Query 중 Tracking Query 부분만 추출
        # track_output = self.layers_track_attention[i](...) → 이전 Frame의 Encoder Feature와 Tracking Query 간의 Attention 수행
        # output = torch.cat([track_output, output[-100:]]) → 이전 Frame 객체 정보와 현재 Query 정보를 결합하여 Decoder에 전달
        # => Multi-Frame Attention 적용

            output = layer(output, memory, tgt_mask=tgt_mask,
                           memory_mask=memory_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask,
                           memory_key_padding_mask=memory_key_padding_mask,
                           pos=pos, query_pos=query_pos)
            if self.return_intermediate:
                intermediate.append(output)

            # => Query를 기반으로 객체를 탐지하고, 이전 Frame 정보를 활용하여 Tracking을 수행

        if self.return_intermediate:
            output = torch.stack(intermediate)

        if self.norm is not None:
            return self.norm(output), output
        return output, output
    
        # return_intermediate=True이면, 모든 Decoder Layer의 출력을 반환
        # self.norm is not None이면 Layer Normalization 적용

`TransformerEncoderLayer`
- TrackFormer의 Transformer Encoder의 한 계층을 구성하는 코드

In [ ]:
class TransformerEncoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super().__init__()

        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        # self.self_attn: Multi-Head Self-Attention Layer → 입력 특징들 간의 관계를 학습습
        # self.linear1 → self.linear2: Feedforward Network (FFN)
        # => Self-Attention을 통해 객체 간 관계를 학습한 후, FFN을 통해 특징을 강화

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    # pos(위치 인코딩)가 주어지면, 입력 Tensor에 위치 인코딩을 추가하여 Transformer가 공간적 정보를 인식할 수 있도록 함
    # => TrackFormer의 Transformer Encoder는 위치 정보를 유지하기 위해 Positional Encoding을 Self-Attention에 직접 전달

    def forward_post(self,
                     src,
                     src_mask: Optional[Tensor] = None,
                     src_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None):
        q = k = self.with_pos_embed(src, pos)
        # 입력에 pos 추가
        src2 = self.self_attn(q, k, value=src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        # MHS 적용
        src = src + self.dropout1(src2)
        # Residual Connection 적용
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        # FFN 적용
        src = src + self.dropout2(src2)
        # Residual Connection 적용
        src = self.norm2(src)
        # Layer Normalization 적용
        return src
        # => Self-Attention 수행

    def forward_pre(self, src,
                    src_mask: Optional[Tensor] = None,
                    src_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None):
        src2 = self.norm1(src)
        # 입력에 먼저 Layer Normalization 적용
        q = k = self.with_pos_embed(src2, pos)
        src2 = self.self_attn(q, k, value=src2, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src2 = self.norm2(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src2))))
        src = src + self.dropout2(src2)
        return src
    
    # forward_post()와 거의 동일하지만, Layer Normalization을 먼저 적용하는 차이점이 있다.
    # => Pre-Norm과 Post-Norm을 선택할 수 있도록 구성

    def forward(self, src,
                src_mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None):
        if self.normalize_before:
            return self.forward_pre(src, src_mask, src_key_padding_mask, pos)
        return self.forward_post(src, src_mask, src_key_padding_mask, pos)
    
    # normalize_before=True이면 forward_pre() 실행 (Pre-Norm)
    # normalize_before=False이면 forward_post() 실행 (Post-Norm)

`TransformerDecoderLayer`
- TrackFormer의 Transformer Decoder의 한 계층을 구성하는 코드

In [ ]:
class TransformerDecoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super().__init__()

        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        # self.self_attn: Multi-Head Self-Attention Layer → Query간의 관계를 학습
        # self.multihead_attn: Cross-Attention Layer → Query와 Encoder Memory간의 관계를 학습
        # self.linear1 → self.linear2: Feedforward Network (FFN)
        # => Query 기반 객체 탐색을 Self-Attention을 통해 수행하고, Cross-Attention을 통해 CNN Feature와 연관성을 학습한 후, FFN을 통해 특징을 강화

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    # pos(위치 인코딩)가 주어지면, 입력 Tensor에 위치 인코딩을 추가하여 Transformer가 공간적 정보를 인식할 수 있도록 함
    # => Transformer Decoder는 Query에 위치 정보를 추가하여 Tracking-by-Attention을 수행

    def forward_post(self, tgt, memory,
                     tgt_mask: Optional[Tensor] = None,
                     memory_mask: Optional[Tensor] = None,
                     tgt_key_padding_mask: Optional[Tensor] = None,
                     memory_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None,
                     query_pos: Optional[Tensor] = None):
        
        q = k = self.with_pos_embed(tgt, query_pos)
        # Query에 pos 추가
        tgt2 = self.self_attn(q, k, value=tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        # MHS 적용
        tgt = tgt + self.dropout1(tgt2)
        # Residual Connection 적용
        tgt = self.norm1(tgt)
        # Layer Normalization 적용
        # => Self-Attention 수행

        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        # Query에 pos 추가 / CNN Feature에 pos 추가 / Query와 CNN Feature간 관계 학습습
        tgt = tgt + self.dropout2(tgt2)
        # Residual Connection 적용
        tgt = self.norm2(tgt)
        # Layer Normalization 적용
        # => Cross-Attention 수행
        
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        # Layer Normalization 적용
        return tgt
        # FFN 수행

    def forward_pre(self, tgt, memory,
                    tgt_mask: Optional[Tensor] = None,
                    memory_mask: Optional[Tensor] = None,
                    tgt_key_padding_mask: Optional[Tensor] = None,
                    memory_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None,
                    query_pos: Optional[Tensor] = None):
        tgt2 = self.norm1(tgt)
        # Layer Normalization 적용
        q = k = self.with_pos_embed(tgt2, query_pos)
        tgt2 = self.self_attn(q, k, value=tgt2, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt2 = self.norm2(tgt)
        # Layer Normalization 적용
        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt2, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt2 = self.norm3(tgt)
        # Layer Normalization 적용
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt2))))
        tgt = tgt + self.dropout3(tgt2)
        return tgt

    # forward_post()와 거의 동일하지만, Layer Normalization을 먼저 적용하는 차이점이 있다.
    # => Pre-Norm과 Post-Norm을 선택할 수 있도록 구성

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        if self.normalize_before:
            return self.forward_pre(tgt, memory, tgt_mask, memory_mask,
                                    tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)
        return self.forward_post(tgt, memory, tgt_mask, memory_mask,
                                 tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)
    
    # normalize_before=True이면 forward_pre() 실행 (Pre-Norm)
    # normalize_before=False이면 forward_post() 실행 (Post-Norm)

`_get_clones`
- 특정 Module을 N개 복사하여 리스트로 반환

`_get_activation_fn(activation)`
- 문자열을 입력으로 받아 해당하는 Activation Function(ReLU,GELU,GLU)을 반환

In [ ]:
def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])
# Transformer의 Encoder와 Decoder가 여러 층으로 구성될 수 있도록 하는 유틸리티 함수

def _get_activation_fn(activation):
    """Return an activation function given a string"""
    if activation == "relu":
        return F.relu
    if activation == "gelu":
        return F.gelu
    if activation == "glu":
        return F.glu
    raise RuntimeError(F"activation should be relu/gelu, not {activation}.")
# Transformer의 Feedforward Network(FFN)에서 사용할 활성화 함수를 선택할 수 있도록 하는 유틸리티 함수

`bulid_transformer`
- TrackFormer에서 사용할 Transformer를 생성하는 함수

In [ ]:
def build_transformer(args):
    return Transformer(
        d_model=args.hidden_dim,
        dropout=args.dropout,
        nhead=args.nheads,
        dim_feedforward=args.dim_feedforward,
        num_encoder_layers=args.enc_layers,
        num_decoder_layers=args.dec_layers,
        normalize_before=args.pre_norm,
        return_intermediate_dec=True,
        track_attention=args.track_attention
    )
# Transformer 모델 생성